In [4]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import json
import pandas as pd

# 定数定義
GRID_SIZE = 3  # 3x3グリッド
BLUE_TEAM_HSV_RANGE = [
    ([95, 150, 150], [125, 255, 255])  # 青チーム
]
RED_TEAM_HSV_RANGE = [
    ([0, 150, 150], [10, 255, 255]),    # 赤チーム（低Hue）
    ([170, 150, 150], [180, 255, 255])  # 赤チーム（高Hue）
]

class MinimapAnalyzer:
    def __init__(self, image_path):
        self.image = cv2.imread(image_path)
        if self.image is None:
            raise ValueError(f"Could not read image from {image_path}")
        self.height, self.width = self.image.shape[:2]
        
        # グリッドのセルサイズを計算
        self.cell_height = self.height // GRID_SIZE
        self.cell_width = self.width // GRID_SIZE
        
        # 分析結果を保存する辞書
        self.analysis_result = {
            "timestamp": datetime.now().isoformat(),
            "image_size": {"width": self.width, "height": self.height},
            "blue_team": [],
            "red_team": [],
            "zone_stats": {}
        }
    
    def detect_team_positions(self, team_color="blue"):
        """チームの位置を検出する"""
        hsv = cv2.cvtColor(self.image, cv2.COLOR_BGR2HSV)
        mask = np.zeros((self.height, self.width), dtype=np.uint8)
        
        # 色範囲の設定
        if team_color == "blue":
            ranges = BLUE_TEAM_HSV_RANGE
        else:
            ranges = RED_TEAM_HSV_RANGE
        
        # マスクの作成
        for (lower, upper) in ranges:
            lower = np.array(lower)
            upper = np.array(upper)
            mask = cv2.bitwise_or(mask, cv2.inRange(hsv, lower, upper))
        
        # ノイズ除去
        mask = cv2.GaussianBlur(mask, (5, 5), 0)
        mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, np.ones((3,3), np.uint8))
        mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, np.ones((3,3), np.uint8))
        
        # 輪郭検出
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        positions = []
        for contour in contours:
            # 面積とアスペクト比でフィルタリング
            area = cv2.contourArea(contour)
            if 50 <= area <= 300:  # プレイヤーアイコンのサイズ範囲
                # 円形度の計算
                perimeter = cv2.arcLength(contour, True)
                circularity = 4 * np.pi * area / (perimeter * perimeter)
                
                if circularity > 0.7:  # 円形度のしきい値
                    M = cv2.moments(contour)
                    if M["m00"] != 0:
                        cx = int(M["m10"] / M["m00"])
                        cy = int(M["m01"] / M["m00"])
                        zone = self.get_zone(cx, cy)
                        positions.append({
                            "position": [cx, cy],
                            "zone": zone
                        })
        
        return positions
    
    def get_zone(self, x, y):
        """座標からゾーン番号を取得する"""
        col = x // self.cell_width
        row = y // self.cell_height
        return row * GRID_SIZE + col + 1
    
    def analyze(self):
        """ミニマップの分析を実行する"""
        # チーム位置の検出
        blue_positions = self.detect_team_positions("blue")
        red_positions = self.detect_team_positions("red")
        
        # プレイヤーIDの割り当て
        for i, pos in enumerate(blue_positions):
            pos["id"] = f"A{i+1}"
        for i, pos in enumerate(red_positions):
            pos["id"] = f"E{i+1}"
        
        # ゾーン統計の計算
        zone_stats = {}
        for zone in range(1, GRID_SIZE * GRID_SIZE + 1):
            blue_count = sum(1 for p in blue_positions if p["zone"] == zone)
            red_count = sum(1 for p in red_positions if p["zone"] == zone)
            zone_stats[str(zone)] = {"blue": blue_count, "red": red_count}
        
        # 結果の保存
        self.analysis_result["blue_team"] = blue_positions
        self.analysis_result["red_team"] = red_positions
        self.analysis_result["zone_stats"] = zone_stats
        
        return self.analysis_result
    
    def visualize(self):
        """分析結果を可視化する"""
        # 元画像のコピー
        vis_img = self.image.copy()
        
        # グリッドの描画
        for i in range(1, GRID_SIZE):
            cv2.line(vis_img, (i * self.cell_width, 0), 
                    (i * self.cell_width, self.height), (255, 255, 255), 1)
            cv2.line(vis_img, (0, i * self.cell_height), 
                    (self.width, i * self.cell_height), (255, 255, 255), 1)
        
        # プレイヤー位置の描画
        for team in ["blue_team", "red_team"]:
            color = (255, 0, 0) if team == "blue_team" else (0, 0, 255)
            for player in self.analysis_result[team]:
                x, y = player["position"]
                cv2.circle(vis_img, (x, y), 5, color, -1)
                cv2.putText(vis_img, player["id"], (x+5, y+5),
                          cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)
        
        # ゾーン統計の描画
        for zone, stats in self.analysis_result["zone_stats"].items():
            zone = int(zone)
            col = (zone - 1) % GRID_SIZE
            row = (zone - 1) // GRID_SIZE
            x = col * self.cell_width + 5
            y = row * self.cell_height + 20
            text = f"B:{stats['blue']} R:{stats['red']}"
            cv2.putText(vis_img, text, (x, y),
                      cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
        
        return vis_img


ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import json
import pandas as pd

# 定数定義
GRID_SIZE = 3  # 3x3グリッド
BLUE_TEAM_HSV_RANGE = [
    ([95, 150, 150], [125, 255, 255])  # 青チーム
]
RED_TEAM_HSV_RANGE = [
    ([0, 150, 150], [10, 255, 255]),    # 赤チーム（低Hue）
    ([170, 150, 150], [180, 255, 255])  # 赤チーム（高Hue）
]

class MinimapAnalyzer:
    def __init__(self, image_path):
        self.image = cv2.imread(image_path)
        if self.image is None:
            raise ValueError(f"Could not read image from {image_path}")
        self.height, self.width = self.image.shape[:2]
        
        # グリッドのセルサイズを計算
        self.cell_height = self.height // GRID_SIZE
        self.cell_width = self.width // GRID_SIZE
        
        # 分析結果を保存する辞書
        self.analysis_result = {
            "timestamp": datetime.now().isoformat(),
            "image_size": {"width": self.width, "height": self.height},
            "blue_team": [],
            "red_team": [],
            "zone_stats": {}
        }
    
    def detect_team_positions(self, team_color="blue"):
        """チームの位置を検出する"""
        hsv = cv2.cvtColor(self.image, cv2.COLOR_BGR2HSV)
        mask = np.zeros((self.height, self.width), dtype=np.uint8)
        
        # 色範囲の設定
        if team_color == "blue":
            ranges = BLUE_TEAM_HSV_RANGE
        else:
            ranges = RED_TEAM_HSV_RANGE
        
        # マスクの作成
        for (lower, upper) in ranges:
            lower = np.array(lower)
            upper = np.array(upper)
            mask = cv2.bitwise_or(mask, cv2.inRange(hsv, lower, upper))
        
        # ノイズ除去
        mask = cv2.GaussianBlur(mask, (5, 5), 0)
        mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, np.ones((3,3), np.uint8))
        mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, np.ones((3,3), np.uint8))
        
        # 輪郭検出
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        positions = []
        for contour in contours:
            # 面積とアスペクト比でフィルタリング
            area = cv2.contourArea(contour)
            if 50 <= area <= 300:  # プレイヤーアイコンのサイズ範囲
                # 円形度の計算
                perimeter = cv2.arcLength(contour, True)
                circularity = 4 * np.pi * area / (perimeter * perimeter)
                
                if circularity > 0.7:  # 円形度のしきい値
                    M = cv2.moments(contour)
                    if M["m00"] != 0:
                        cx = int(M["m10"] / M["m00"])
                        cy = int(M["m01"] / M["m00"])
                        zone = self.get_zone(cx, cy)
                        positions.append({
                            "position": [cx, cy],
                            "zone": zone
                        })
        
        return positions
    
    def get_zone(self, x, y):
        """座標からゾーン番号を取得する"""
        col = x // self.cell_width
        row = y // self.cell_height
        return row * GRID_SIZE + col + 1
    
    def analyze(self):
        """ミニマップの分析を実行する"""
        # チーム位置の検出
        blue_positions = self.detect_team_positions("blue")
        red_positions = self.detect_team_positions("red")
        
        # プレイヤーIDの割り当て
        for i, pos in enumerate(blue_positions):
            pos["id"] = f"A{i+1}"
        for i, pos in enumerate(red_positions):
            pos["id"] = f"E{i+1}"
        
        # ゾーン統計の計算
        zone_stats = {}
        for zone in range(1, GRID_SIZE * GRID_SIZE + 1):
            blue_count = sum(1 for p in blue_positions if p["zone"] == zone)
            red_count = sum(1 for p in red_positions if p["zone"] == zone)
            zone_stats[str(zone)] = {"blue": blue_count, "red": red_count}
        
        # 結果の保存
        self.analysis_result["blue_team"] = blue_positions
        self.analysis_result["red_team"] = red_positions
        self.analysis_result["zone_stats"] = zone_stats
        
        return self.analysis_result
    
    def visualize(self):
        """分析結果を可視化する"""
        # 元画像のコピー
        vis_img = self.image.copy()
        
        # グリッドの描画
        for i in range(1, GRID_SIZE):
            cv2.line(vis_img, (i * self.cell_width, 0), 
                    (i * self.cell_width, self.height), (255, 255, 255), 1)
            cv2.line(vis_img, (0, i * self.cell_height), 
                    (self.width, i * self.cell_height), (255, 255, 255), 1)
        
        # プレイヤー位置の描画
        for team in ["blue_team", "red_team"]:
            color = (255, 0, 0) if team == "blue_team" else (0, 0, 255)
            for player in self.analysis_result[team]:
                x, y = player["position"]
                cv2.circle(vis_img, (x, y), 5, color, -1)
                cv2.putText(vis_img, player["id"], (x+5, y+5),
                          cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)
        
        # ゾーン統計の描画
        for zone, stats in self.analysis_result["zone_stats"].items():
            zone = int(zone)
            col = (zone - 1) % GRID_SIZE
            row = (zone - 1) // GRID_SIZE
            x = col * self.cell_width + 5
            y = row * self.cell_height + 20
            text = f"B:{stats['blue']} R:{stats['red']}"
            cv2.putText(vis_img, text, (x, y),
                      cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
        
        return vis_img


ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import json
import pandas as pd

# 定数定義
GRID_SIZE = 3  # 3x3グリッド
BLUE_TEAM_HSV_RANGE = [
    ([95, 150, 150], [125, 255, 255])  # 青チーム
]
RED_TEAM_HSV_RANGE = [
    ([0, 150, 150], [10, 255, 255]),    # 赤チーム（低Hue）
    ([170, 150, 150], [180, 255, 255])  # 赤チーム（高Hue）
]

class MinimapAnalyzer:
    def __init__(self, image_path):
        self.image = cv2.imread(image_path)
        if self.image is None:
            raise ValueError(f"Could not read image from {image_path}")
        self.height, self.width = self.image.shape[:2]
        
        # グリッドのセルサイズを計算
        self.cell_height = self.height // GRID_SIZE
        self.cell_width = self.width // GRID_SIZE
        
        # 分析結果を保存する辞書
        self.analysis_result = {
            "timestamp": datetime.now().isoformat(),
            "image_size": {"width": self.width, "height": self.height},
            "blue_team": [],
            "red_team": [],
            "zone_stats": {}
        }
    
    def detect_team_positions(self, team_color="blue"):
        """チームの位置を検出する"""
        hsv = cv2.cvtColor(self.image, cv2.COLOR_BGR2HSV)
        mask = np.zeros((self.height, self.width), dtype=np.uint8)
        
        # 色範囲の設定
        if team_color == "blue":
            ranges = BLUE_TEAM_HSV_RANGE
        else:
            ranges = RED_TEAM_HSV_RANGE
        
        # マスクの作成
        for (lower, upper) in ranges:
            lower = np.array(lower)
            upper = np.array(upper)
            mask = cv2.bitwise_or(mask, cv2.inRange(hsv, lower, upper))
        
        # ノイズ除去
        mask = cv2.GaussianBlur(mask, (5, 5), 0)
        mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, np.ones((3,3), np.uint8))
        mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, np.ones((3,3), np.uint8))
        
        # 輪郭検出
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        positions = []
        for contour in contours:
            # 面積とアスペクト比でフィルタリング
            area = cv2.contourArea(contour)
            if 50 <= area <= 300:  # プレイヤーアイコンのサイズ範囲
                # 円形度の計算
                perimeter = cv2.arcLength(contour, True)
                circularity = 4 * np.pi * area / (perimeter * perimeter)
                
                if circularity > 0.7:  # 円形度のしきい値
                    M = cv2.moments(contour)
                    if M["m00"] != 0:
                        cx = int(M["m10"] / M["m00"])
                        cy = int(M["m01"] / M["m00"])
                        zone = self.get_zone(cx, cy)
                        positions.append({
                            "position": [cx, cy],
                            "zone": zone
                        })
        
        return positions
    
    def get_zone(self, x, y):
        """座標からゾーン番号を取得する"""
        col = x // self.cell_width
        row = y // self.cell_height
        return row * GRID_SIZE + col + 1
    
    def analyze(self):
        """ミニマップの分析を実行する"""
        # チーム位置の検出
        blue_positions = self.detect_team_positions("blue")
        red_positions = self.detect_team_positions("red")
        
        # プレイヤーIDの割り当て
        for i, pos in enumerate(blue_positions):
            pos["id"] = f"A{i+1}"
        for i, pos in enumerate(red_positions):
            pos["id"] = f"E{i+1}"
        
        # ゾーン統計の計算
        zone_stats = {}
        for zone in range(1, GRID_SIZE * GRID_SIZE + 1):
            blue_count = sum(1 for p in blue_positions if p["zone"] == zone)
            red_count = sum(1 for p in red_positions if p["zone"] == zone)
            zone_stats[str(zone)] = {"blue": blue_count, "red": red_count}
        
        # 結果の保存
        self.analysis_result["blue_team"] = blue_positions
        self.analysis_result["red_team"] = red_positions
        self.analysis_result["zone_stats"] = zone_stats
        
        return self.analysis_result
    
    def visualize(self):
        """分析結果を可視化する"""
        # 元画像のコピー
        vis_img = self.image.copy()
        
        # グリッドの描画
        for i in range(1, GRID_SIZE):
            cv2.line(vis_img, (i * self.cell_width, 0), 
                    (i * self.cell_width, self.height), (255, 255, 255), 1)
            cv2.line(vis_img, (0, i * self.cell_height), 
                    (self.width, i * self.cell_height), (255, 255, 255), 1)
        
        # プレイヤー位置の描画
        for team in ["blue_team", "red_team"]:
            color = (255, 0, 0) if team == "blue_team" else (0, 0, 255)
            for player in self.analysis_result[team]:
                x, y = player["position"]
                cv2.circle(vis_img, (x, y), 5, color, -1)
                cv2.putText(vis_img, player["id"], (x+5, y+5),
                          cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)
        
        # ゾーン統計の描画
        for zone, stats in self.analysis_result["zone_stats"].items():
            zone = int(zone)
            col = (zone - 1) % GRID_SIZE
            row = (zone - 1) // GRID_SIZE
            x = col * self.cell_width + 5
            y = row * self.cell_height + 20
            text = f"B:{stats['blue']} R:{stats['red']}"
            cv2.putText(vis_img, text, (x, y),
                      cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
        
        return vis_img


ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import json
import pandas as pd

# 定数定義
GRID_SIZE = 3  # 3x3グリッド
BLUE_TEAM_HSV_RANGE = [
    ([95, 150, 150], [125, 255, 255])  # 青チーム
]
RED_TEAM_HSV_RANGE = [
    ([0, 150, 150], [10, 255, 255]),    # 赤チーム（低Hue）
    ([170, 150, 150], [180, 255, 255])  # 赤チーム（高Hue）
]

class MinimapAnalyzer:
    def __init__(self, image_path):
        self.image = cv2.imread(image_path)
        if self.image is None:
            raise ValueError(f"Could not read image from {image_path}")
        self.height, self.width = self.image.shape[:2]
        
        # グリッドのセルサイズを計算
        self.cell_height = self.height // GRID_SIZE
        self.cell_width = self.width // GRID_SIZE
        
        # 分析結果を保存する辞書
        self.analysis_result = {
            "timestamp": datetime.now().isoformat(),
            "image_size": {"width": self.width, "height": self.height},
            "blue_team": [],
            "red_team": [],
            "zone_stats": {}
        }
    
    def detect_team_positions(self, team_color="blue"):
        """チームの位置を検出する"""
        hsv = cv2.cvtColor(self.image, cv2.COLOR_BGR2HSV)
        mask = np.zeros((self.height, self.width), dtype=np.uint8)
        
        # 色範囲の設定
        if team_color == "blue":
            ranges = BLUE_TEAM_HSV_RANGE
        else:
            ranges = RED_TEAM_HSV_RANGE
        
        # マスクの作成
        for (lower, upper) in ranges:
            lower = np.array(lower)
            upper = np.array(upper)
            mask = cv2.bitwise_or(mask, cv2.inRange(hsv, lower, upper))
        
        # ノイズ除去
        mask = cv2.GaussianBlur(mask, (5, 5), 0)
        mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, np.ones((3,3), np.uint8))
        mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, np.ones((3,3), np.uint8))
        
        # 輪郭検出
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        positions = []
        for contour in contours:
            # 面積とアスペクト比でフィルタリング
            area = cv2.contourArea(contour)
            if 50 <= area <= 300:  # プレイヤーアイコンのサイズ範囲
                # 円形度の計算
                perimeter = cv2.arcLength(contour, True)
                circularity = 4 * np.pi * area / (perimeter * perimeter)
                
                if circularity > 0.7:  # 円形度のしきい値
                    M = cv2.moments(contour)
                    if M["m00"] != 0:
                        cx = int(M["m10"] / M["m00"])
                        cy = int(M["m01"] / M["m00"])
                        zone = self.get_zone(cx, cy)
                        positions.append({
                            "position": [cx, cy],
                            "zone": zone
                        })
        
        return positions
    
    def get_zone(self, x, y):
        """座標からゾーン番号を取得する"""
        col = x // self.cell_width
        row = y // self.cell_height
        return row * GRID_SIZE + col + 1
    
    def analyze(self):
        """ミニマップの分析を実行する"""
        # チーム位置の検出
        blue_positions = self.detect_team_positions("blue")
        red_positions = self.detect_team_positions("red")
        
        # プレイヤーIDの割り当て
        for i, pos in enumerate(blue_positions):
            pos["id"] = f"A{i+1}"
        for i, pos in enumerate(red_positions):
            pos["id"] = f"E{i+1}"
        
        # ゾーン統計の計算
        zone_stats = {}
        for zone in range(1, GRID_SIZE * GRID_SIZE + 1):
            blue_count = sum(1 for p in blue_positions if p["zone"] == zone)
            red_count = sum(1 for p in red_positions if p["zone"] == zone)
            zone_stats[str(zone)] = {"blue": blue_count, "red": red_count}
        
        # 結果の保存
        self.analysis_result["blue_team"] = blue_positions
        self.analysis_result["red_team"] = red_positions
        self.analysis_result["zone_stats"] = zone_stats
        
        return self.analysis_result
    
    def visualize(self):
        """分析結果を可視化する"""
        # 元画像のコピー
        vis_img = self.image.copy()
        
        # グリッドの描画
        for i in range(1, GRID_SIZE):
            cv2.line(vis_img, (i * self.cell_width, 0), 
                    (i * self.cell_width, self.height), (255, 255, 255), 1)
            cv2.line(vis_img, (0, i * self.cell_height), 
                    (self.width, i * self.cell_height), (255, 255, 255), 1)
        
        # プレイヤー位置の描画
        for team in ["blue_team", "red_team"]:
            color = (255, 0, 0) if team == "blue_team" else (0, 0, 255)
            for player in self.analysis_result[team]:
                x, y = player["position"]
                cv2.circle(vis_img, (x, y), 5, color, -1)
                cv2.putText(vis_img, player["id"], (x+5, y+5),
                          cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)
        
        # ゾーン統計の描画
        for zone, stats in self.analysis_result["zone_stats"].items():
            zone = int(zone)
            col = (zone - 1) % GRID_SIZE
            row = (zone - 1) // GRID_SIZE
            x = col * self.cell_width + 5
            y = row * self.cell_height + 20
            text = f"B:{stats['blue']} R:{stats['red']}"
            cv2.putText(vis_img, text, (x, y),
                      cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
        
        return vis_img


ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import json
import pandas as pd

# 定数定義
GRID_SIZE = 3  # 3x3グリッド
BLUE_TEAM_HSV_RANGE = [
    ([95, 150, 150], [125, 255, 255])  # 青チーム
]
RED_TEAM_HSV_RANGE = [
    ([0, 150, 150], [10, 255, 255]),    # 赤チーム（低Hue）
    ([170, 150, 150], [180, 255, 255])  # 赤チーム（高Hue）
]

class MinimapAnalyzer:
    def __init__(self, image_path):
        self.image = cv2.imread(image_path)
        if self.image is None:
            raise ValueError(f"Could not read image from {image_path}")
        self.height, self.width = self.image.shape[:2]
        
        # グリッドのセルサイズを計算
        self.cell_height = self.height // GRID_SIZE
        self.cell_width = self.width // GRID_SIZE
        
        # 分析結果を保存する辞書
        self.analysis_result = {
            "timestamp": datetime.now().isoformat(),
            "image_size": {"width": self.width, "height": self.height},
            "blue_team": [],
            "red_team": [],
            "zone_stats": {}
        }
    
    def detect_team_positions(self, team_color="blue"):
        """チームの位置を検出する"""
        hsv = cv2.cvtColor(self.image, cv2.COLOR_BGR2HSV)
        mask = np.zeros((self.height, self.width), dtype=np.uint8)
        
        # 色範囲の設定
        if team_color == "blue":
            ranges = BLUE_TEAM_HSV_RANGE
        else:
            ranges = RED_TEAM_HSV_RANGE
        
        # マスクの作成
        for (lower, upper) in ranges:
            lower = np.array(lower)
            upper = np.array(upper)
            mask = cv2.bitwise_or(mask, cv2.inRange(hsv, lower, upper))
        
        # ノイズ除去
        mask = cv2.GaussianBlur(mask, (5, 5), 0)
        mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, np.ones((3,3), np.uint8))
        mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, np.ones((3,3), np.uint8))
        
        # 輪郭検出
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        positions = []
        for contour in contours:
            # 面積とアスペクト比でフィルタリング
            area = cv2.contourArea(contour)
            if 50 <= area <= 300:  # プレイヤーアイコンのサイズ範囲
                # 円形度の計算
                perimeter = cv2.arcLength(contour, True)
                circularity = 4 * np.pi * area / (perimeter * perimeter)
                
                if circularity > 0.7:  # 円形度のしきい値
                    M = cv2.moments(contour)
                    if M["m00"] != 0:
                        cx = int(M["m10"] / M["m00"])
                        cy = int(M["m01"] / M["m00"])
                        zone = self.get_zone(cx, cy)
                        positions.append({
                            "position": [cx, cy],
                            "zone": zone
                        })
        
        return positions
    
    def get_zone(self, x, y):
        """座標からゾーン番号を取得する"""
        col = x // self.cell_width
        row = y // self.cell_height
        return row * GRID_SIZE + col + 1
    
    def analyze(self):
        """ミニマップの分析を実行する"""
        # チーム位置の検出
        blue_positions = self.detect_team_positions("blue")
        red_positions = self.detect_team_positions("red")
        
        # プレイヤーIDの割り当て
        for i, pos in enumerate(blue_positions):
            pos["id"] = f"A{i+1}"
        for i, pos in enumerate(red_positions):
            pos["id"] = f"E{i+1}"
        
        # ゾーン統計の計算
        zone_stats = {}
        for zone in range(1, GRID_SIZE * GRID_SIZE + 1):
            blue_count = sum(1 for p in blue_positions if p["zone"] == zone)
            red_count = sum(1 for p in red_positions if p["zone"] == zone)
            zone_stats[str(zone)] = {"blue": blue_count, "red": red_count}
        
        # 結果の保存
        self.analysis_result["blue_team"] = blue_positions
        self.analysis_result["red_team"] = red_positions
        self.analysis_result["zone_stats"] = zone_stats
        
        return self.analysis_result
    
    def visualize(self):
        """分析結果を可視化する"""
        # 元画像のコピー
        vis_img = self.image.copy()
        
        # グリッドの描画
        for i in range(1, GRID_SIZE):
            cv2.line(vis_img, (i * self.cell_width, 0), 
                    (i * self.cell_width, self.height), (255, 255, 255), 1)
            cv2.line(vis_img, (0, i * self.cell_height), 
                    (self.width, i * self.cell_height), (255, 255, 255), 1)
        
        # プレイヤー位置の描画
        for team in ["blue_team", "red_team"]:
            color = (255, 0, 0) if team == "blue_team" else (0, 0, 255)
            for player in self.analysis_result[team]:
                x, y = player["position"]
                cv2.circle(vis_img, (x, y), 5, color, -1)
                cv2.putText(vis_img, player["id"], (x+5, y+5),
                          cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)
        
        # ゾーン統計の描画
        for zone, stats in self.analysis_result["zone_stats"].items():
            zone = int(zone)
            col = (zone - 1) % GRID_SIZE
            row = (zone - 1) // GRID_SIZE
            x = col * self.cell_width + 5
            y = row * self.cell_height + 20
            text = f"B:{stats['blue']} R:{stats['red']}"
            cv2.putText(vis_img, text, (x, y),
                      cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
        
        return vis_img


ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import json
import pandas as pd

# 定数定義
GRID_SIZE = 3  # 3x3グリッド
BLUE_TEAM_HSV_RANGE = [
    ([95, 150, 150], [125, 255, 255])  # 青チーム
]
RED_TEAM_HSV_RANGE = [
    ([0, 150, 150], [10, 255, 255]),    # 赤チーム（低Hue）
    ([170, 150, 150], [180, 255, 255])  # 赤チーム（高Hue）
]

class MinimapAnalyzer:
    def __init__(self, image_path):
        self.image = cv2.imread(image_path)
        if self.image is None:
            raise ValueError(f"Could not read image from {image_path}")
        self.height, self.width = self.image.shape[:2]
        
        # グリッドのセルサイズを計算
        self.cell_height = self.height // GRID_SIZE
        self.cell_width = self.width // GRID_SIZE
        
        # 分析結果を保存する辞書
        self.analysis_result = {
            "timestamp": datetime.now().isoformat(),
            "image_size": {"width": self.width, "height": self.height},
            "blue_team": [],
            "red_team": [],
            "zone_stats": {}
        }
    
    def detect_team_positions(self, team_color="blue"):
        """チームの位置を検出する"""
        hsv = cv2.cvtColor(self.image, cv2.COLOR_BGR2HSV)
        mask = np.zeros((self.height, self.width), dtype=np.uint8)
        
        # 色範囲の設定
        if team_color == "blue":
            ranges = BLUE_TEAM_HSV_RANGE
        else:
            ranges = RED_TEAM_HSV_RANGE
        
        # マスクの作成
        for (lower, upper) in ranges:
            lower = np.array(lower)
            upper = np.array(upper)
            mask = cv2.bitwise_or(mask, cv2.inRange(hsv, lower, upper))
        
        # ノイズ除去
        mask = cv2.GaussianBlur(mask, (5, 5), 0)
        mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, np.ones((3,3), np.uint8))
        mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, np.ones((3,3), np.uint8))
        
        # 輪郭検出
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        positions = []
        for contour in contours:
            # 面積とアスペクト比でフィルタリング
            area = cv2.contourArea(contour)
            if 50 <= area <= 300:  # プレイヤーアイコンのサイズ範囲
                # 円形度の計算
                perimeter = cv2.arcLength(contour, True)
                circularity = 4 * np.pi * area / (perimeter * perimeter)
                
                if circularity > 0.7:  # 円形度のしきい値
                    M = cv2.moments(contour)
                    if M["m00"] != 0:
                        cx = int(M["m10"] / M["m00"])
                        cy = int(M["m01"] / M["m00"])
                        zone = self.get_zone(cx, cy)
                        positions.append({
                            "position": [cx, cy],
                            "zone": zone
                        })
        
        return positions
    
    def get_zone(self, x, y):
        """座標からゾーン番号を取得する"""
        col = x // self.cell_width
        row = y // self.cell_height
        return row * GRID_SIZE + col + 1
    
    def analyze(self):
        """ミニマップの分析を実行する"""
        # チーム位置の検出
        blue_positions = self.detect_team_positions("blue")
        red_positions = self.detect_team_positions("red")
        
        # プレイヤーIDの割り当て
        for i, pos in enumerate(blue_positions):
            pos["id"] = f"A{i+1}"
        for i, pos in enumerate(red_positions):
            pos["id"] = f"E{i+1}"
        
        # ゾーン統計の計算
        zone_stats = {}
        for zone in range(1, GRID_SIZE * GRID_SIZE + 1):
            blue_count = sum(1 for p in blue_positions if p["zone"] == zone)
            red_count = sum(1 for p in red_positions if p["zone"] == zone)
            zone_stats[str(zone)] = {"blue": blue_count, "red": red_count}
        
        # 結果の保存
        self.analysis_result["blue_team"] = blue_positions
        self.analysis_result["red_team"] = red_positions
        self.analysis_result["zone_stats"] = zone_stats
        
        return self.analysis_result
    
    def visualize(self):
        """分析結果を可視化する"""
        # 元画像のコピー
        vis_img = self.image.copy()
        
        # グリッドの描画
        for i in range(1, GRID_SIZE):
            cv2.line(vis_img, (i * self.cell_width, 0), 
                    (i * self.cell_width, self.height), (255, 255, 255), 1)
            cv2.line(vis_img, (0, i * self.cell_height), 
                    (self.width, i * self.cell_height), (255, 255, 255), 1)
        
        # プレイヤー位置の描画
        for team in ["blue_team", "red_team"]:
            color = (255, 0, 0) if team == "blue_team" else (0, 0, 255)
            for player in self.analysis_result[team]:
                x, y = player["position"]
                cv2.circle(vis_img, (x, y), 5, color, -1)
                cv2.putText(vis_img, player["id"], (x+5, y+5),
                          cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)
        
        # ゾーン統計の描画
        for zone, stats in self.analysis_result["zone_stats"].items():
            zone = int(zone)
            col = (zone - 1) % GRID_SIZE
            row = (zone - 1) // GRID_SIZE
            x = col * self.cell_width + 5
            y = row * self.cell_height + 20
            text = f"B:{stats['blue']} R:{stats['red']}"
            cv2.putText(vis_img, text, (x, y),
                      cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
        
        return vis_img


ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import json
import pandas as pd

# 定数定義
GRID_SIZE = 3  # 3x3グリッド
BLUE_TEAM_HSV_RANGE = [
    ([95, 150, 150], [125, 255, 255])  # 青チーム
]
RED_TEAM_HSV_RANGE = [
    ([0, 150, 150], [10, 255, 255]),    # 赤チーム（低Hue）
    ([170, 150, 150], [180, 255, 255])  # 赤チーム（高Hue）
]

class MinimapAnalyzer:
    def __init__(self, image_path):
        self.image = cv2.imread(image_path)
        if self.image is None:
            raise ValueError(f"Could not read image from {image_path}")
        self.height, self.width = self.image.shape[:2]
        
        # グリッドのセルサイズを計算
        self.cell_height = self.height // GRID_SIZE
        self.cell_width = self.width // GRID_SIZE
        
        # 分析結果を保存する辞書
        self.analysis_result = {
            "timestamp": datetime.now().isoformat(),
            "image_size": {"width": self.width, "height": self.height},
            "blue_team": [],
            "red_team": [],
            "zone_stats": {}
        }
    
    def detect_team_positions(self, team_color="blue"):
        """チームの位置を検出する"""
        hsv = cv2.cvtColor(self.image, cv2.COLOR_BGR2HSV)
        mask = np.zeros((self.height, self.width), dtype=np.uint8)
        
        # 色範囲の設定
        if team_color == "blue":
            ranges = BLUE_TEAM_HSV_RANGE
        else:
            ranges = RED_TEAM_HSV_RANGE
        
        # マスクの作成
        for (lower, upper) in ranges:
            lower = np.array(lower)
            upper = np.array(upper)
            mask = cv2.bitwise_or(mask, cv2.inRange(hsv, lower, upper))
        
        # ノイズ除去
        mask = cv2.GaussianBlur(mask, (5, 5), 0)
        mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, np.ones((3,3), np.uint8))
        mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, np.ones((3,3), np.uint8))
        
        # 輪郭検出
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        positions = []
        for contour in contours:
            # 面積とアスペクト比でフィルタリング
            area = cv2.contourArea(contour)
            if 50 <= area <= 300:  # プレイヤーアイコンのサイズ範囲
                # 円形度の計算
                perimeter = cv2.arcLength(contour, True)
                circularity = 4 * np.pi * area / (perimeter * perimeter)
                
                if circularity > 0.7:  # 円形度のしきい値
                    M = cv2.moments(contour)
                    if M["m00"] != 0:
                        cx = int(M["m10"] / M["m00"])
                        cy = int(M["m01"] / M["m00"])
                        zone = self.get_zone(cx, cy)
                        positions.append({
                            "position": [cx, cy],
                            "zone": zone
                        })
        
        return positions
    
    def get_zone(self, x, y):
        """座標からゾーン番号を取得する"""
        col = x // self.cell_width
        row = y // self.cell_height
        return row * GRID_SIZE + col + 1
    
    def analyze(self):
        """ミニマップの分析を実行する"""
        # チーム位置の検出
        blue_positions = self.detect_team_positions("blue")
        red_positions = self.detect_team_positions("red")
        
        # プレイヤーIDの割り当て
        for i, pos in enumerate(blue_positions):
            pos["id"] = f"A{i+1}"
        for i, pos in enumerate(red_positions):
            pos["id"] = f"E{i+1}"
        
        # ゾーン統計の計算
        zone_stats = {}
        for zone in range(1, GRID_SIZE * GRID_SIZE + 1):
            blue_count = sum(1 for p in blue_positions if p["zone"] == zone)
            red_count = sum(1 for p in red_positions if p["zone"] == zone)
            zone_stats[str(zone)] = {"blue": blue_count, "red": red_count}
        
        # 結果の保存
        self.analysis_result["blue_team"] = blue_positions
        self.analysis_result["red_team"] = red_positions
        self.analysis_result["zone_stats"] = zone_stats
        
        return self.analysis_result
    
    def visualize(self):
        """分析結果を可視化する"""
        # 元画像のコピー
        vis_img = self.image.copy()
        
        # グリッドの描画
        for i in range(1, GRID_SIZE):
            cv2.line(vis_img, (i * self.cell_width, 0), 
                    (i * self.cell_width, self.height), (255, 255, 255), 1)
            cv2.line(vis_img, (0, i * self.cell_height), 
                    (self.width, i * self.cell_height), (255, 255, 255), 1)
        
        # プレイヤー位置の描画
        for team in ["blue_team", "red_team"]:
            color = (255, 0, 0) if team == "blue_team" else (0, 0, 255)
            for player in self.analysis_result[team]:
                x, y = player["position"]
                cv2.circle(vis_img, (x, y), 5, color, -1)
                cv2.putText(vis_img, player["id"], (x+5, y+5),
                          cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)
        
        # ゾーン統計の描画
        for zone, stats in self.analysis_result["zone_stats"].items():
            zone = int(zone)
            col = (zone - 1) % GRID_SIZE
            row = (zone - 1) // GRID_SIZE
            x = col * self.cell_width + 5
            y = row * self.cell_height + 20
            text = f"B:{stats['blue']} R:{stats['red']}"
            cv2.putText(vis_img, text, (x, y),
                      cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
        
        return vis_img


ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import json
import pandas as pd

# 定数定義
GRID_SIZE = 3  # 3x3グリッド
BLUE_TEAM_HSV_RANGE = [
    ([95, 150, 150], [125, 255, 255])  # 青チーム
]
RED_TEAM_HSV_RANGE = [
    ([0, 150, 150], [10, 255, 255]),    # 赤チーム（低Hue）
    ([170, 150, 150], [180, 255, 255])  # 赤チーム（高Hue）
]

class MinimapAnalyzer:
    def __init__(self, image_path):
        self.image = cv2.imread(image_path)
        if self.image is None:
            raise ValueError(f"Could not read image from {image_path}")
        self.height, self.width = self.image.shape[:2]
        
        # グリッドのセルサイズを計算
        self.cell_height = self.height // GRID_SIZE
        self.cell_width = self.width // GRID_SIZE
        
        # 分析結果を保存する辞書
        self.analysis_result = {
            "timestamp": datetime.now().isoformat(),
            "image_size": {"width": self.width, "height": self.height},
            "blue_team": [],
            "red_team": [],
            "zone_stats": {}
        }
    
    def detect_team_positions(self, team_color="blue"):
        """チームの位置を検出する"""
        hsv = cv2.cvtColor(self.image, cv2.COLOR_BGR2HSV)
        mask = np.zeros((self.height, self.width), dtype=np.uint8)
        
        # 色範囲の設定
        if team_color == "blue":
            ranges = BLUE_TEAM_HSV_RANGE
        else:
            ranges = RED_TEAM_HSV_RANGE
        
        # マスクの作成
        for (lower, upper) in ranges:
            lower = np.array(lower)
            upper = np.array(upper)
            mask = cv2.bitwise_or(mask, cv2.inRange(hsv, lower, upper))
        
        # ノイズ除去
        mask = cv2.GaussianBlur(mask, (5, 5), 0)
        mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, np.ones((3,3), np.uint8))
        mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, np.ones((3,3), np.uint8))
        
        # 輪郭検出
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        positions = []
        for contour in contours:
            # 面積とアスペクト比でフィルタリング
            area = cv2.contourArea(contour)
            if 50 <= area <= 300:  # プレイヤーアイコンのサイズ範囲
                # 円形度の計算
                perimeter = cv2.arcLength(contour, True)
                circularity = 4 * np.pi * area / (perimeter * perimeter)
                
                if circularity > 0.7:  # 円形度のしきい値
                    M = cv2.moments(contour)
                    if M["m00"] != 0:
                        cx = int(M["m10"] / M["m00"])
                        cy = int(M["m01"] / M["m00"])
                        zone = self.get_zone(cx, cy)
                        positions.append({
                            "position": [cx, cy],
                            "zone": zone
                        })
        
        return positions
    
    def get_zone(self, x, y):
        """座標からゾーン番号を取得する"""
        col = x // self.cell_width
        row = y // self.cell_height
        return row * GRID_SIZE + col + 1
    
    def analyze(self):
        """ミニマップの分析を実行する"""
        # チーム位置の検出
        blue_positions = self.detect_team_positions("blue")
        red_positions = self.detect_team_positions("red")
        
        # プレイヤーIDの割り当て
        for i, pos in enumerate(blue_positions):
            pos["id"] = f"A{i+1}"
        for i, pos in enumerate(red_positions):
            pos["id"] = f"E{i+1}"
        
        # ゾーン統計の計算
        zone_stats = {}
        for zone in range(1, GRID_SIZE * GRID_SIZE + 1):
            blue_count = sum(1 for p in blue_positions if p["zone"] == zone)
            red_count = sum(1 for p in red_positions if p["zone"] == zone)
            zone_stats[str(zone)] = {"blue": blue_count, "red": red_count}
        
        # 結果の保存
        self.analysis_result["blue_team"] = blue_positions
        self.analysis_result["red_team"] = red_positions
        self.analysis_result["zone_stats"] = zone_stats
        
        return self.analysis_result
    
    def visualize(self):
        """分析結果を可視化する"""
        # 元画像のコピー
        vis_img = self.image.copy()
        
        # グリッドの描画
        for i in range(1, GRID_SIZE):
            cv2.line(vis_img, (i * self.cell_width, 0), 
                    (i * self.cell_width, self.height), (255, 255, 255), 1)
            cv2.line(vis_img, (0, i * self.cell_height), 
                    (self.width, i * self.cell_height), (255, 255, 255), 1)
        
        # プレイヤー位置の描画
        for team in ["blue_team", "red_team"]:
            color = (255, 0, 0) if team == "blue_team" else (0, 0, 255)
            for player in self.analysis_result[team]:
                x, y = player["position"]
                cv2.circle(vis_img, (x, y), 5, color, -1)
                cv2.putText(vis_img, player["id"], (x+5, y+5),
                          cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)
        
        # ゾーン統計の描画
        for zone, stats in self.analysis_result["zone_stats"].items():
            zone = int(zone)
            col = (zone - 1) % GRID_SIZE
            row = (zone - 1) // GRID_SIZE
            x = col * self.cell_width + 5
            y = row * self.cell_height + 20
            text = f"B:{stats['blue']} R:{stats['red']}"
            cv2.putText(vis_img, text, (x, y),
                      cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
        
        return vis_img


ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import json
import pandas as pd

# 定数定義
GRID_SIZE = 3  # 3x3グリッド
BLUE_TEAM_HSV_RANGE = [
    ([95, 150, 150], [125, 255, 255])  # 青チーム
]
RED_TEAM_HSV_RANGE = [
    ([0, 150, 150], [10, 255, 255]),    # 赤チーム（低Hue）
    ([170, 150, 150], [180, 255, 255])  # 赤チーム（高Hue）
]

class MinimapAnalyzer:
    def __init__(self, image_path):
        self.image = cv2.imread(image_path)
        if self.image is None:
            raise ValueError(f"Could not read image from {image_path}")
        self.height, self.width = self.image.shape[:2]
        
        # グリッドのセルサイズを計算
        self.cell_height = self.height // GRID_SIZE
        self.cell_width = self.width // GRID_SIZE
        
        # 分析結果を保存する辞書
        self.analysis_result = {
            "timestamp": datetime.now().isoformat(),
            "image_size": {"width": self.width, "height": self.height},
            "blue_team": [],
            "red_team": [],
            "zone_stats": {}
        }
    
    def detect_team_positions(self, team_color="blue"):
        """チームの位置を検出する"""
        hsv = cv2.cvtColor(self.image, cv2.COLOR_BGR2HSV)
        mask = np.zeros((self.height, self.width), dtype=np.uint8)
        
        # 色範囲の設定
        if team_color == "blue":
            ranges = BLUE_TEAM_HSV_RANGE
        else:
            ranges = RED_TEAM_HSV_RANGE
        
        # マスクの作成
        for (lower, upper) in ranges:
            lower = np.array(lower)
            upper = np.array(upper)
            mask = cv2.bitwise_or(mask, cv2.inRange(hsv, lower, upper))
        
        # ノイズ除去
        mask = cv2.GaussianBlur(mask, (5, 5), 0)
        mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, np.ones((3,3), np.uint8))
        mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, np.ones((3,3), np.uint8))
        
        # 輪郭検出
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        positions = []
        for contour in contours:
            # 面積とアスペクト比でフィルタリング
            area = cv2.contourArea(contour)
            if 50 <= area <= 300:  # プレイヤーアイコンのサイズ範囲
                # 円形度の計算
                perimeter = cv2.arcLength(contour, True)
                circularity = 4 * np.pi * area / (perimeter * perimeter)
                
                if circularity > 0.7:  # 円形度のしきい値
                    M = cv2.moments(contour)
                    if M["m00"] != 0:
                        cx = int(M["m10"] / M["m00"])
                        cy = int(M["m01"] / M["m00"])
                        zone = self.get_zone(cx, cy)
                        positions.append({
                            "position": [cx, cy],
                            "zone": zone
                        })
        
        return positions
    
    def get_zone(self, x, y):
        """座標からゾーン番号を取得する"""
        col = x // self.cell_width
        row = y // self.cell_height
        return row * GRID_SIZE + col + 1
    
    def analyze(self):
        """ミニマップの分析を実行する"""
        # チーム位置の検出
        blue_positions = self.detect_team_positions("blue")
        red_positions = self.detect_team_positions("red")
        
        # プレイヤーIDの割り当て
        for i, pos in enumerate(blue_positions):
            pos["id"] = f"A{i+1}"
        for i, pos in enumerate(red_positions):
            pos["id"] = f"E{i+1}"
        
        # ゾーン統計の計算
        zone_stats = {}
        for zone in range(1, GRID_SIZE * GRID_SIZE + 1):
            blue_count = sum(1 for p in blue_positions if p["zone"] == zone)
            red_count = sum(1 for p in red_positions if p["zone"] == zone)
            zone_stats[str(zone)] = {"blue": blue_count, "red": red_count}
        
        # 結果の保存
        self.analysis_result["blue_team"] = blue_positions
        self.analysis_result["red_team"] = red_positions
        self.analysis_result["zone_stats"] = zone_stats
        
        return self.analysis_result
    
    def visualize(self):
        """分析結果を可視化する"""
        # 元画像のコピー
        vis_img = self.image.copy()
        
        # グリッドの描画
        for i in range(1, GRID_SIZE):
            cv2.line(vis_img, (i * self.cell_width, 0), 
                    (i * self.cell_width, self.height), (255, 255, 255), 1)
            cv2.line(vis_img, (0, i * self.cell_height), 
                    (self.width, i * self.cell_height), (255, 255, 255), 1)
        
        # プレイヤー位置の描画
        for team in ["blue_team", "red_team"]:
            color = (255, 0, 0) if team == "blue_team" else (0, 0, 255)
            for player in self.analysis_result[team]:
                x, y = player["position"]
                cv2.circle(vis_img, (x, y), 5, color, -1)
                cv2.putText(vis_img, player["id"], (x+5, y+5),
                          cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)
        
        # ゾーン統計の描画
        for zone, stats in self.analysis_result["zone_stats"].items():
            zone = int(zone)
            col = (zone - 1) % GRID_SIZE
            row = (zone - 1) // GRID_SIZE
            x = col * self.cell_width + 5
            y = row * self.cell_height + 20
            text = f"B:{stats['blue']} R:{stats['red']}"
            cv2.putText(vis_img, text, (x, y),
                      cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
        
        return vis_img


ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import json
import pandas as pd

# 定数定義
GRID_SIZE = 3  # 3x3グリッド
BLUE_TEAM_HSV_RANGE = [
    ([95, 150, 150], [125, 255, 255])  # 青チーム
]
RED_TEAM_HSV_RANGE = [
    ([0, 150, 150], [10, 255, 255]),    # 赤チーム（低Hue）
    ([170, 150, 150], [180, 255, 255])  # 赤チーム（高Hue）
]

class MinimapAnalyzer:
    def __init__(self, image_path):
        self.image = cv2.imread(image_path)
        if self.image is None:
            raise ValueError(f"Could not read image from {image_path}")
        self.height, self.width = self.image.shape[:2]
        
        # グリッドのセルサイズを計算
        self.cell_height = self.height // GRID_SIZE
        self.cell_width = self.width // GRID_SIZE
        
        # 分析結果を保存する辞書
        self.analysis_result = {
            "timestamp": datetime.now().isoformat(),
            "image_size": {"width": self.width, "height": self.height},
            "blue_team": [],
            "red_team": [],
            "zone_stats": {}
        }
    
    def detect_team_positions(self, team_color="blue"):
        """チームの位置を検出する"""
        hsv = cv2.cvtColor(self.image, cv2.COLOR_BGR2HSV)
        mask = np.zeros((self.height, self.width), dtype=np.uint8)
        
        # 色範囲の設定
        if team_color == "blue":
            ranges = BLUE_TEAM_HSV_RANGE
        else:
            ranges = RED_TEAM_HSV_RANGE
        
        # マスクの作成
        for (lower, upper) in ranges:
            lower = np.array(lower)
            upper = np.array(upper)
            mask = cv2.bitwise_or(mask, cv2.inRange(hsv, lower, upper))
        
        # ノイズ除去
        mask = cv2.GaussianBlur(mask, (5, 5), 0)
        mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, np.ones((3,3), np.uint8))
        mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, np.ones((3,3), np.uint8))
        
        # 輪郭検出
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        positions = []
        for contour in contours:
            # 面積とアスペクト比でフィルタリング
            area = cv2.contourArea(contour)
            if 50 <= area <= 300:  # プレイヤーアイコンのサイズ範囲
                # 円形度の計算
                perimeter = cv2.arcLength(contour, True)
                circularity = 4 * np.pi * area / (perimeter * perimeter)
                
                if circularity > 0.7:  # 円形度のしきい値
                    M = cv2.moments(contour)
                    if M["m00"] != 0:
                        cx = int(M["m10"] / M["m00"])
                        cy = int(M["m01"] / M["m00"])
                        zone = self.get_zone(cx, cy)
                        positions.append({
                            "position": [cx, cy],
                            "zone": zone
                        })
        
        return positions
    
    def get_zone(self, x, y):
        """座標からゾーン番号を取得する"""
        col = x // self.cell_width
        row = y // self.cell_height
        return row * GRID_SIZE + col + 1
    
    def analyze(self):
        """ミニマップの分析を実行する"""
        # チーム位置の検出
        blue_positions = self.detect_team_positions("blue")
        red_positions = self.detect_team_positions("red")
        
        # プレイヤーIDの割り当て
        for i, pos in enumerate(blue_positions):
            pos["id"] = f"A{i+1}"
        for i, pos in enumerate(red_positions):
            pos["id"] = f"E{i+1}"
        
        # ゾーン統計の計算
        zone_stats = {}
        for zone in range(1, GRID_SIZE * GRID_SIZE + 1):
            blue_count = sum(1 for p in blue_positions if p["zone"] == zone)
            red_count = sum(1 for p in red_positions if p["zone"] == zone)
            zone_stats[str(zone)] = {"blue": blue_count, "red": red_count}
        
        # 結果の保存
        self.analysis_result["blue_team"] = blue_positions
        self.analysis_result["red_team"] = red_positions
        self.analysis_result["zone_stats"] = zone_stats
        
        return self.analysis_result
    
    def visualize(self):
        """分析結果を可視化する"""
        # 元画像のコピー
        vis_img = self.image.copy()
        
        # グリッドの描画
        for i in range(1, GRID_SIZE):
            cv2.line(vis_img, (i * self.cell_width, 0), 
                    (i * self.cell_width, self.height), (255, 255, 255), 1)
            cv2.line(vis_img, (0, i * self.cell_height), 
                    (self.width, i * self.cell_height), (255, 255, 255), 1)
        
        # プレイヤー位置の描画
        for team in ["blue_team", "red_team"]:
            color = (255, 0, 0) if team == "blue_team" else (0, 0, 255)
            for player in self.analysis_result[team]:
                x, y = player["position"]
                cv2.circle(vis_img, (x, y), 5, color, -1)
                cv2.putText(vis_img, player["id"], (x+5, y+5),
                          cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)
        
        # ゾーン統計の描画
        for zone, stats in self.analysis_result["zone_stats"].items():
            zone = int(zone)
            col = (zone - 1) % GRID_SIZE
            row = (zone - 1) // GRID_SIZE
            x = col * self.cell_width + 5
            y = row * self.cell_height + 20
            text = f"B:{stats['blue']} R:{stats['red']}"
            cv2.putText(vis_img, text, (x, y),
                      cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
        
        return vis_img


ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import json
import pandas as pd

# 定数定義
GRID_SIZE = 3  # 3x3グリッド
BLUE_TEAM_HSV_RANGE = [
    ([95, 150, 150], [125, 255, 255])  # 青チーム
]
RED_TEAM_HSV_RANGE = [
    ([0, 150, 150], [10, 255, 255]),    # 赤チーム（低Hue）
    ([170, 150, 150], [180, 255, 255])  # 赤チーム（高Hue）
]

class MinimapAnalyzer:
    def __init__(self, image_path):
        self.image = cv2.imread(image_path)
        if self.image is None:
            raise ValueError(f"Could not read image from {image_path}")
        self.height, self.width = self.image.shape[:2]
        
        # グリッドのセルサイズを計算
        self.cell_height = self.height // GRID_SIZE
        self.cell_width = self.width // GRID_SIZE
        
        # 分析結果を保存する辞書
        self.analysis_result = {
            "timestamp": datetime.now().isoformat(),
            "image_size": {"width": self.width, "height": self.height},
            "blue_team": [],
            "red_team": [],
            "zone_stats": {}
        }
    
    def detect_team_positions(self, team_color="blue"):
        """チームの位置を検出する"""
        hsv = cv2.cvtColor(self.image, cv2.COLOR_BGR2HSV)
        mask = np.zeros((self.height, self.width), dtype=np.uint8)
        
        # 色範囲の設定
        if team_color == "blue":
            ranges = BLUE_TEAM_HSV_RANGE
        else:
            ranges = RED_TEAM_HSV_RANGE
        
        # マスクの作成
        for (lower, upper) in ranges:
            lower = np.array(lower)
            upper = np.array(upper)
            mask = cv2.bitwise_or(mask, cv2.inRange(hsv, lower, upper))
        
        # ノイズ除去
        mask = cv2.GaussianBlur(mask, (5, 5), 0)
        mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, np.ones((3,3), np.uint8))
        mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, np.ones((3,3), np.uint8))
        
        # 輪郭検出
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        positions = []
        for contour in contours:
            # 面積とアスペクト比でフィルタリング
            area = cv2.contourArea(contour)
            if 50 <= area <= 300:  # プレイヤーアイコンのサイズ範囲
                # 円形度の計算
                perimeter = cv2.arcLength(contour, True)
                circularity = 4 * np.pi * area / (perimeter * perimeter)
                
                if circularity > 0.7:  # 円形度のしきい値
                    M = cv2.moments(contour)
                    if M["m00"] != 0:
                        cx = int(M["m10"] / M["m00"])
                        cy = int(M["m01"] / M["m00"])
                        zone = self.get_zone(cx, cy)
                        positions.append({
                            "position": [cx, cy],
                            "zone": zone
                        })
        
        return positions
    
    def get_zone(self, x, y):
        """座標からゾーン番号を取得する"""
        col = x // self.cell_width
        row = y // self.cell_height
        return row * GRID_SIZE + col + 1
    
    def analyze(self):
        """ミニマップの分析を実行する"""
        # チーム位置の検出
        blue_positions = self.detect_team_positions("blue")
        red_positions = self.detect_team_positions("red")
        
        # プレイヤーIDの割り当て
        for i, pos in enumerate(blue_positions):
            pos["id"] = f"A{i+1}"
        for i, pos in enumerate(red_positions):
            pos["id"] = f"E{i+1}"
        
        # ゾーン統計の計算
        zone_stats = {}
        for zone in range(1, GRID_SIZE * GRID_SIZE + 1):
            blue_count = sum(1 for p in blue_positions if p["zone"] == zone)
            red_count = sum(1 for p in red_positions if p["zone"] == zone)
            zone_stats[str(zone)] = {"blue": blue_count, "red": red_count}
        
        # 結果の保存
        self.analysis_result["blue_team"] = blue_positions
        self.analysis_result["red_team"] = red_positions
        self.analysis_result["zone_stats"] = zone_stats
        
        return self.analysis_result
    
    def visualize(self):
        """分析結果を可視化する"""
        # 元画像のコピー
        vis_img = self.image.copy()
        
        # グリッドの描画
        for i in range(1, GRID_SIZE):
            cv2.line(vis_img, (i * self.cell_width, 0), 
                    (i * self.cell_width, self.height), (255, 255, 255), 1)
            cv2.line(vis_img, (0, i * self.cell_height), 
                    (self.width, i * self.cell_height), (255, 255, 255), 1)
        
        # プレイヤー位置の描画
        for team in ["blue_team", "red_team"]:
            color = (255, 0, 0) if team == "blue_team" else (0, 0, 255)
            for player in self.analysis_result[team]:
                x, y = player["position"]
                cv2.circle(vis_img, (x, y), 5, color, -1)
                cv2.putText(vis_img, player["id"], (x+5, y+5),
                          cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)
        
        # ゾーン統計の描画
        for zone, stats in self.analysis_result["zone_stats"].items():
            zone = int(zone)
            col = (zone - 1) % GRID_SIZE
            row = (zone - 1) // GRID_SIZE
            x = col * self.cell_width + 5
            y = row * self.cell_height + 20
            text = f"B:{stats['blue']} R:{stats['red']}"
            cv2.putText(vis_img, text, (x, y),
                      cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
        
        return vis_img


ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import json
import pandas as pd

# 定数定義
GRID_SIZE = 3  # 3x3グリッド
BLUE_TEAM_HSV_RANGE = [
    ([95, 150, 150], [125, 255, 255])  # 青チーム
]
RED_TEAM_HSV_RANGE = [
    ([0, 150, 150], [10, 255, 255]),    # 赤チーム（低Hue）
    ([170, 150, 150], [180, 255, 255])  # 赤チーム（高Hue）
]

class MinimapAnalyzer:
    def __init__(self, image_path):
        self.image = cv2.imread(image_path)
        if self.image is None:
            raise ValueError(f"Could not read image from {image_path}")
        self.height, self.width = self.image.shape[:2]
        
        # グリッドのセルサイズを計算
        self.cell_height = self.height // GRID_SIZE
        self.cell_width = self.width // GRID_SIZE
        
        # 分析結果を保存する辞書
        self.analysis_result = {
            "timestamp": datetime.now().isoformat(),
            "image_size": {"width": self.width, "height": self.height},
            "blue_team": [],
            "red_team": [],
            "zone_stats": {}
        }
    
    def detect_team_positions(self, team_color="blue"):
        """チームの位置を検出する"""
        hsv = cv2.cvtColor(self.image, cv2.COLOR_BGR2HSV)
        mask = np.zeros((self.height, self.width), dtype=np.uint8)
        
        # 色範囲の設定
        if team_color == "blue":
            ranges = BLUE_TEAM_HSV_RANGE
        else:
            ranges = RED_TEAM_HSV_RANGE
        
        # マスクの作成
        for (lower, upper) in ranges:
            lower = np.array(lower)
            upper = np.array(upper)
            mask = cv2.bitwise_or(mask, cv2.inRange(hsv, lower, upper))
        
        # ノイズ除去
        mask = cv2.GaussianBlur(mask, (5, 5), 0)
        mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, np.ones((3,3), np.uint8))
        mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, np.ones((3,3), np.uint8))
        
        # 輪郭検出
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        positions = []
        for contour in contours:
            # 面積とアスペクト比でフィルタリング
            area = cv2.contourArea(contour)
            if 50 <= area <= 300:  # プレイヤーアイコンのサイズ範囲
                # 円形度の計算
                perimeter = cv2.arcLength(contour, True)
                circularity = 4 * np.pi * area / (perimeter * perimeter)
                
                if circularity > 0.7:  # 円形度のしきい値
                    M = cv2.moments(contour)
                    if M["m00"] != 0:
                        cx = int(M["m10"] / M["m00"])
                        cy = int(M["m01"] / M["m00"])
                        zone = self.get_zone(cx, cy)
                        positions.append({
                            "position": [cx, cy],
                            "zone": zone
                        })
        
        return positions
    
    def get_zone(self, x, y):
        """座標からゾーン番号を取得する"""
        col = x // self.cell_width
        row = y // self.cell_height
        return row * GRID_SIZE + col + 1
    
    def analyze(self):
        """ミニマップの分析を実行する"""
        # チーム位置の検出
        blue_positions = self.detect_team_positions("blue")
        red_positions = self.detect_team_positions("red")
        
        # プレイヤーIDの割り当て
        for i, pos in enumerate(blue_positions):
            pos["id"] = f"A{i+1}"
        for i, pos in enumerate(red_positions):
            pos["id"] = f"E{i+1}"
        
        # ゾーン統計の計算
        zone_stats = {}
        for zone in range(1, GRID_SIZE * GRID_SIZE + 1):
            blue_count = sum(1 for p in blue_positions if p["zone"] == zone)
            red_count = sum(1 for p in red_positions if p["zone"] == zone)
            zone_stats[str(zone)] = {"blue": blue_count, "red": red_count}
        
        # 結果の保存
        self.analysis_result["blue_team"] = blue_positions
        self.analysis_result["red_team"] = red_positions
        self.analysis_result["zone_stats"] = zone_stats
        
        return self.analysis_result
    
    def visualize(self):
        """分析結果を可視化する"""
        # 元画像のコピー
        vis_img = self.image.copy()
        
        # グリッドの描画
        for i in range(1, GRID_SIZE):
            cv2.line(vis_img, (i * self.cell_width, 0), 
                    (i * self.cell_width, self.height), (255, 255, 255), 1)
            cv2.line(vis_img, (0, i * self.cell_height), 
                    (self.width, i * self.cell_height), (255, 255, 255), 1)
        
        # プレイヤー位置の描画
        for team in ["blue_team", "red_team"]:
            color = (255, 0, 0) if team == "blue_team" else (0, 0, 255)
            for player in self.analysis_result[team]:
                x, y = player["position"]
                cv2.circle(vis_img, (x, y), 5, color, -1)
                cv2.putText(vis_img, player["id"], (x+5, y+5),
                          cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)
        
        # ゾーン統計の描画
        for zone, stats in self.analysis_result["zone_stats"].items():
            zone = int(zone)
            col = (zone - 1) % GRID_SIZE
            row = (zone - 1) // GRID_SIZE
            x = col * self.cell_width + 5
            y = row * self.cell_height + 20
            text = f"B:{stats['blue']} R:{stats['red']}"
            cv2.putText(vis_img, text, (x, y),
                      cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
        
        return vis_img


ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import json
import pandas as pd

# 定数定義
GRID_SIZE = 3  # 3x3グリッド
BLUE_TEAM_HSV_RANGE = [
    ([95, 150, 150], [125, 255, 255])  # 青チーム
]
RED_TEAM_HSV_RANGE = [
    ([0, 150, 150], [10, 255, 255]),    # 赤チーム（低Hue）
    ([170, 150, 150], [180, 255, 255])  # 赤チーム（高Hue）
]

class MinimapAnalyzer:
    def __init__(self, image_path):
        self.image = cv2.imread(image_path)
        if self.image is None:
            raise ValueError(f"Could not read image from {image_path}")
        self.height, self.width = self.image.shape[:2]
        
        # グリッドのセルサイズを計算
        self.cell_height = self.height // GRID_SIZE
        self.cell_width = self.width // GRID_SIZE
        
        # 分析結果を保存する辞書
        self.analysis_result = {
            "timestamp": datetime.now().isoformat(),
            "image_size": {"width": self.width, "height": self.height},
            "blue_team": [],
            "red_team": [],
            "zone_stats": {}
        }
    
    def detect_team_positions(self, team_color="blue"):
        """チームの位置を検出する"""
        hsv = cv2.cvtColor(self.image, cv2.COLOR_BGR2HSV)
        mask = np.zeros((self.height, self.width), dtype=np.uint8)
        
        # 色範囲の設定
        if team_color == "blue":
            ranges = BLUE_TEAM_HSV_RANGE
        else:
            ranges = RED_TEAM_HSV_RANGE
        
        # マスクの作成
        for (lower, upper) in ranges:
            lower = np.array(lower)
            upper = np.array(upper)
            mask = cv2.bitwise_or(mask, cv2.inRange(hsv, lower, upper))
        
        # ノイズ除去
        mask = cv2.GaussianBlur(mask, (5, 5), 0)
        mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, np.ones((3,3), np.uint8))
        mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, np.ones((3,3), np.uint8))
        
        # 輪郭検出
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        positions = []
        for contour in contours:
            # 面積とアスペクト比でフィルタリング
            area = cv2.contourArea(contour)
            if 50 <= area <= 300:  # プレイヤーアイコンのサイズ範囲
                # 円形度の計算
                perimeter = cv2.arcLength(contour, True)
                circularity = 4 * np.pi * area / (perimeter * perimeter)
                
                if circularity > 0.7:  # 円形度のしきい値
                    M = cv2.moments(contour)
                    if M["m00"] != 0:
                        cx = int(M["m10"] / M["m00"])
                        cy = int(M["m01"] / M["m00"])
                        zone = self.get_zone(cx, cy)
                        positions.append({
                            "position": [cx, cy],
                            "zone": zone
                        })
        
        return positions
    
    def get_zone(self, x, y):
        """座標からゾーン番号を取得する"""
        col = x // self.cell_width
        row = y // self.cell_height
        return row * GRID_SIZE + col + 1
    
    def analyze(self):
        """ミニマップの分析を実行する"""
        # チーム位置の検出
        blue_positions = self.detect_team_positions("blue")
        red_positions = self.detect_team_positions("red")
        
        # プレイヤーIDの割り当て
        for i, pos in enumerate(blue_positions):
            pos["id"] = f"A{i+1}"
        for i, pos in enumerate(red_positions):
            pos["id"] = f"E{i+1}"
        
        # ゾーン統計の計算
        zone_stats = {}
        for zone in range(1, GRID_SIZE * GRID_SIZE + 1):
            blue_count = sum(1 for p in blue_positions if p["zone"] == zone)
            red_count = sum(1 for p in red_positions if p["zone"] == zone)
            zone_stats[str(zone)] = {"blue": blue_count, "red": red_count}
        
        # 結果の保存
        self.analysis_result["blue_team"] = blue_positions
        self.analysis_result["red_team"] = red_positions
        self.analysis_result["zone_stats"] = zone_stats
        
        return self.analysis_result
    
    def visualize(self):
        """分析結果を可視化する"""
        # 元画像のコピー
        vis_img = self.image.copy()
        
        # グリッドの描画
        for i in range(1, GRID_SIZE):
            cv2.line(vis_img, (i * self.cell_width, 0), 
                    (i * self.cell_width, self.height), (255, 255, 255), 1)
            cv2.line(vis_img, (0, i * self.cell_height), 
                    (self.width, i * self.cell_height), (255, 255, 255), 1)
        
        # プレイヤー位置の描画
        for team in ["blue_team", "red_team"]:
            color = (255, 0, 0) if team == "blue_team" else (0, 0, 255)
            for player in self.analysis_result[team]:
                x, y = player["position"]
                cv2.circle(vis_img, (x, y), 5, color, -1)
                cv2.putText(vis_img, player["id"], (x+5, y+5),
                          cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)
        
        # ゾーン統計の描画
        for zone, stats in self.analysis_result["zone_stats"].items():
            zone = int(zone)
            col = (zone - 1) % GRID_SIZE
            row = (zone - 1) // GRID_SIZE
            x = col * self.cell_width + 5
            y = row * self.cell_height + 20
            text = f"B:{stats['blue']} R:{stats['red']}"
            cv2.putText(vis_img, text, (x, y),
                      cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
        
        return vis_img


ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import json
import pandas as pd

# 定数定義
GRID_SIZE = 3  # 3x3グリッド
BLUE_TEAM_HSV_RANGE = [
    ([95, 150, 150], [125, 255, 255])  # 青チーム
]
RED_TEAM_HSV_RANGE = [
    ([0, 150, 150], [10, 255, 255]),    # 赤チーム（低Hue）
    ([170, 150, 150], [180, 255, 255])  # 赤チーム（高Hue）
]

class MinimapAnalyzer:
    def __init__(self, image_path):
        self.image = cv2.imread(image_path)
        if self.image is None:
            raise ValueError(f"Could not read image from {image_path}")
        self.height, self.width = self.image.shape[:2]
        
        # グリッドのセルサイズを計算
        self.cell_height = self.height // GRID_SIZE
        self.cell_width = self.width // GRID_SIZE
        
        # 分析結果を保存する辞書
        self.analysis_result = {
            "timestamp": datetime.now().isoformat(),
            "image_size": {"width": self.width, "height": self.height},
            "blue_team": [],
            "red_team": [],
            "zone_stats": {}
        }
    
    def detect_team_positions(self, team_color="blue"):
        """チームの位置を検出する"""
        hsv = cv2.cvtColor(self.image, cv2.COLOR_BGR2HSV)
        mask = np.zeros((self.height, self.width), dtype=np.uint8)
        
        # 色範囲の設定
        if team_color == "blue":
            ranges = BLUE_TEAM_HSV_RANGE
        else:
            ranges = RED_TEAM_HSV_RANGE
        
        # マスクの作成
        for (lower, upper) in ranges:
            lower = np.array(lower)
            upper = np.array(upper)
            mask = cv2.bitwise_or(mask, cv2.inRange(hsv, lower, upper))
        
        # ノイズ除去
        mask = cv2.GaussianBlur(mask, (5, 5), 0)
        mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, np.ones((3,3), np.uint8))
        mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, np.ones((3,3), np.uint8))
        
        # 輪郭検出
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        positions = []
        for contour in contours:
            # 面積とアスペクト比でフィルタリング
            area = cv2.contourArea(contour)
            if 50 <= area <= 300:  # プレイヤーアイコンのサイズ範囲
                # 円形度の計算
                perimeter = cv2.arcLength(contour, True)
                circularity = 4 * np.pi * area / (perimeter * perimeter)
                
                if circularity > 0.7:  # 円形度のしきい値
                    M = cv2.moments(contour)
                    if M["m00"] != 0:
                        cx = int(M["m10"] / M["m00"])
                        cy = int(M["m01"] / M["m00"])
                        zone = self.get_zone(cx, cy)
                        positions.append({
                            "position": [cx, cy],
                            "zone": zone
                        })
        
        return positions
    
    def get_zone(self, x, y):
        """座標からゾーン番号を取得する"""
        col = x // self.cell_width
        row = y // self.cell_height
        return row * GRID_SIZE + col + 1
    
    def analyze(self):
        """ミニマップの分析を実行する"""
        # チーム位置の検出
        blue_positions = self.detect_team_positions("blue")
        red_positions = self.detect_team_positions("red")
        
        # プレイヤーIDの割り当て
        for i, pos in enumerate(blue_positions):
            pos["id"] = f"A{i+1}"
        for i, pos in enumerate(red_positions):
            pos["id"] = f"E{i+1}"
        
        # ゾーン統計の計算
        zone_stats = {}
        for zone in range(1, GRID_SIZE * GRID_SIZE + 1):
            blue_count = sum(1 for p in blue_positions if p["zone"] == zone)
            red_count = sum(1 for p in red_positions if p["zone"] == zone)
            zone_stats[str(zone)] = {"blue": blue_count, "red": red_count}
        
        # 結果の保存
        self.analysis_result["blue_team"] = blue_positions
        self.analysis_result["red_team"] = red_positions
        self.analysis_result["zone_stats"] = zone_stats
        
        return self.analysis_result
    
    def visualize(self):
        """分析結果を可視化する"""
        # 元画像のコピー
        vis_img = self.image.copy()
        
        # グリッドの描画
        for i in range(1, GRID_SIZE):
            cv2.line(vis_img, (i * self.cell_width, 0), 
                    (i * self.cell_width, self.height), (255, 255, 255), 1)
            cv2.line(vis_img, (0, i * self.cell_height), 
                    (self.width, i * self.cell_height), (255, 255, 255), 1)
        
        # プレイヤー位置の描画
        for team in ["blue_team", "red_team"]:
            color = (255, 0, 0) if team == "blue_team" else (0, 0, 255)
            for player in self.analysis_result[team]:
                x, y = player["position"]
                cv2.circle(vis_img, (x, y), 5, color, -1)
                cv2.putText(vis_img, player["id"], (x+5, y+5),
                          cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)
        
        # ゾーン統計の描画
        for zone, stats in self.analysis_result["zone_stats"].items():
            zone = int(zone)
            col = (zone - 1) % GRID_SIZE
            row = (zone - 1) // GRID_SIZE
            x = col * self.cell_width + 5
            y = row * self.cell_height + 20
            text = f"B:{stats['blue']} R:{stats['red']}"
            cv2.putText(vis_img, text, (x, y),
                      cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
        
        return vis_img


ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import json
import pandas as pd

# 定数定義
GRID_SIZE = 3  # 3x3グリッド
BLUE_TEAM_HSV_RANGE = [
    ([95, 150, 150], [125, 255, 255])  # 青チーム
]
RED_TEAM_HSV_RANGE = [
    ([0, 150, 150], [10, 255, 255]),    # 赤チーム（低Hue）
    ([170, 150, 150], [180, 255, 255])  # 赤チーム（高Hue）
]

class MinimapAnalyzer:
    def __init__(self, image_path):
        self.image = cv2.imread(image_path)
        if self.image is None:
            raise ValueError(f"Could not read image from {image_path}")
        self.height, self.width = self.image.shape[:2]
        
        # グリッドのセルサイズを計算
        self.cell_height = self.height // GRID_SIZE
        self.cell_width = self.width // GRID_SIZE
        
        # 分析結果を保存する辞書
        self.analysis_result = {
            "timestamp": datetime.now().isoformat(),
            "image_size": {"width": self.width, "height": self.height},
            "blue_team": [],
            "red_team": [],
            "zone_stats": {}
        }
    
    def detect_team_positions(self, team_color="blue"):
        """チームの位置を検出する"""
        hsv = cv2.cvtColor(self.image, cv2.COLOR_BGR2HSV)
        mask = np.zeros((self.height, self.width), dtype=np.uint8)
        
        # 色範囲の設定
        if team_color == "blue":
            ranges = BLUE_TEAM_HSV_RANGE
        else:
            ranges = RED_TEAM_HSV_RANGE
        
        # マスクの作成
        for (lower, upper) in ranges:
            lower = np.array(lower)
            upper = np.array(upper)
            mask = cv2.bitwise_or(mask, cv2.inRange(hsv, lower, upper))
        
        # ノイズ除去
        mask = cv2.GaussianBlur(mask, (5, 5), 0)
        mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, np.ones((3,3), np.uint8))
        mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, np.ones((3,3), np.uint8))
        
        # 輪郭検出
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        positions = []
        for contour in contours:
            # 面積とアスペクト比でフィルタリング
            area = cv2.contourArea(contour)
            if 50 <= area <= 300:  # プレイヤーアイコンのサイズ範囲
                # 円形度の計算
                perimeter = cv2.arcLength(contour, True)
                circularity = 4 * np.pi * area / (perimeter * perimeter)
                
                if circularity > 0.7:  # 円形度のしきい値
                    M = cv2.moments(contour)
                    if M["m00"] != 0:
                        cx = int(M["m10"] / M["m00"])
                        cy = int(M["m01"] / M["m00"])
                        zone = self.get_zone(cx, cy)
                        positions.append({
                            "position": [cx, cy],
                            "zone": zone
                        })
        
        return positions
    
    def get_zone(self, x, y):
        """座標からゾーン番号を取得する"""
        col = x // self.cell_width
        row = y // self.cell_height
        return row * GRID_SIZE + col + 1
    
    def analyze(self):
        """ミニマップの分析を実行する"""
        # チーム位置の検出
        blue_positions = self.detect_team_positions("blue")
        red_positions = self.detect_team_positions("red")
        
        # プレイヤーIDの割り当て
        for i, pos in enumerate(blue_positions):
            pos["id"] = f"A{i+1}"
        for i, pos in enumerate(red_positions):
            pos["id"] = f"E{i+1}"
        
        # ゾーン統計の計算
        zone_stats = {}
        for zone in range(1, GRID_SIZE * GRID_SIZE + 1):
            blue_count = sum(1 for p in blue_positions if p["zone"] == zone)
            red_count = sum(1 for p in red_positions if p["zone"] == zone)
            zone_stats[str(zone)] = {"blue": blue_count, "red": red_count}
        
        # 結果の保存
        self.analysis_result["blue_team"] = blue_positions
        self.analysis_result["red_team"] = red_positions
        self.analysis_result["zone_stats"] = zone_stats
        
        return self.analysis_result
    
    def visualize(self):
        """分析結果を可視化する"""
        # 元画像のコピー
        vis_img = self.image.copy()
        
        # グリッドの描画
        for i in range(1, GRID_SIZE):
            cv2.line(vis_img, (i * self.cell_width, 0), 
                    (i * self.cell_width, self.height), (255, 255, 255), 1)
            cv2.line(vis_img, (0, i * self.cell_height), 
                    (self.width, i * self.cell_height), (255, 255, 255), 1)
        
        # プレイヤー位置の描画
        for team in ["blue_team", "red_team"]:
            color = (255, 0, 0) if team == "blue_team" else (0, 0, 255)
            for player in self.analysis_result[team]:
                x, y = player["position"]
                cv2.circle(vis_img, (x, y), 5, color, -1)
                cv2.putText(vis_img, player["id"], (x+5, y+5),
                          cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)
        
        # ゾーン統計の描画
        for zone, stats in self.analysis_result["zone_stats"].items():
            zone = int(zone)
            col = (zone - 1) % GRID_SIZE
            row = (zone - 1) // GRID_SIZE
            x = col * self.cell_width + 5
            y = row * self.cell_height + 20
            text = f"B:{stats['blue']} R:{stats['red']}"
            cv2.putText(vis_img, text, (x, y),
                      cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
        
        return vis_img


ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
# ベース画像のパスを設定
base_image_path = "../base_picture/IMG_0062.jpg"

# MinimapAnalyzerのインスタンスを作成
analyzer = MinimapAnalyzer(base_image_path)

# 分析を実行
result = analyzer.analyze()

# 結果を可視化
vis_img = analyzer.visualize()

# 結果の表示
plt.figure(figsize=(12, 6))

# 元画像と分析結果を並べて表示
plt.subplot(121)
plt.imshow(cv2.cvtColor(analyzer.image, cv2.COLOR_BGR2RGB))
plt.title("Original Image")
plt.axis("off")

plt.subplot(122)
plt.imshow(cv2.cvtColor(vis_img, cv2.COLOR_BGR2RGB))
plt.title("Analysis Result")
plt.axis("off")

plt.tight_layout()
plt.show()

# 分析結果の詳細を表示
print("\n=== Analysis Results ===")
print("\nBlue Team Positions:")
for player in result["blue_team"]:
    print(f"Player {player['id']}: Position {player['position']}, Zone {player['zone']}")

print("\nRed Team Positions:")
for player in result["red_team"]:
    print(f"Player {player['id']}: Position {player['position']}, Zone {player['zone']}")

print("\nZone Statistics:")
for zone, stats in result["zone_stats"].items():
    print(f"Zone {zone}: Blue Team: {stats['blue']}, Red Team: {stats['red']}")

# 結果をJSONファイルとして保存
output_path = "minimap_analysis_result.json"
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(result, f, indent=2, ensure_ascii=False)
print(f"\nResults saved to {output_path}")


In [ ]:
# 時系列データの分析用の関数を追加

class TimeSeriesAnalyzer:
    def __init__(self):
        self.history = []
        
    def add_frame(self, frame_result):
        """フレームの分析結果を履歴に追加"""
        self.history.append({
            "timestamp": frame_result["timestamp"],
            "blue_team": frame_result["blue_team"],
            "red_team": frame_result["red_team"],
            "zone_stats": frame_result["zone_stats"]
        })
    
    def calculate_team_centroid(self, team_positions):
        """チームの重心を計算"""
        if not team_positions:
            return None
        x_coords = [p["position"][0] for p in team_positions]
        y_coords = [p["position"][1] for p in team_positions]
        return [sum(x_coords) / len(x_coords), sum(y_coords) / len(y_coords)]
    
    def calculate_team_spread(self, team_positions, centroid):
        """チームの分散度を計算"""
        if not team_positions or not centroid:
            return None
        distances = [np.sqrt((p["position"][0] - centroid[0])**2 + 
                           (p["position"][1] - centroid[1])**2) 
                    for p in team_positions]
        return sum(distances) / len(distances)
    
    def get_zone_control_history(self):
        """ゾーン支配率の履歴を取得"""
        zone_control = []
        for frame in self.history:
            frame_control = {}
            for zone, stats in frame["zone_stats"].items():
                total = stats["blue"] + stats["red"]
                if total > 0:
                    frame_control[zone] = {
                        "blue_control": stats["blue"] / total,
                        "red_control": stats["red"] / total
                    }
                else:
                    frame_control[zone] = {
                        "blue_control": 0,
                        "red_control": 0
                    }
            zone_control.append({
                "timestamp": frame["timestamp"],
                "zone_control": frame_control
            })
        return zone_control
    
    def get_team_movement_stats(self):
        """チームの移動統計を取得"""
        movement_stats = []
        for frame in self.history:
            blue_centroid = self.calculate_team_centroid(frame["blue_team"])
            red_centroid = self.calculate_team_centroid(frame["red_team"])
            
            stats = {
                "timestamp": frame["timestamp"],
                "blue_team": {
                    "centroid": blue_centroid,
                    "spread": self.calculate_team_spread(frame["blue_team"], blue_centroid),
                    "player_count": len(frame["blue_team"])
                },
                "red_team": {
                    "centroid": red_centroid,
                    "spread": self.calculate_team_spread(frame["red_team"], red_centroid),
                    "player_count": len(frame["red_team"])
                }
            }
            movement_stats.append(stats)
        return movement_stats

# 時系列分析の実行（サンプルとして現在のフレームを追加）
ts_analyzer = TimeSeriesAnalyzer()
ts_analyzer.add_frame(result)

# 移動統計の取得と表示
movement_stats = ts_analyzer.get_team_movement_stats()
print("\n=== Team Movement Statistics ===")
for stats in movement_stats:
    print(f"\nTimestamp: {stats['timestamp']}")
    print("Blue Team:")
    print(f"  Centroid: {stats['blue_team']['centroid']}")
    print(f"  Spread: {stats['blue_team']['spread']}")
    print(f"  Player Count: {stats['blue_team']['player_count']}")
    print("Red Team:")
    print(f"  Centroid: {stats['red_team']['centroid']}")
    print(f"  Spread: {stats['red_team']['spread']}")
    print(f"  Player Count: {stats['red_team']['player_count']}")

# ゾーン支配率の取得と表示
zone_control = ts_analyzer.get_zone_control_history()
print("\n=== Zone Control History ===")
for control in zone_control:
    print(f"\nTimestamp: {control['timestamp']}")
    for zone, stats in control["zone_control"].items():
        print(f"Zone {zone}:")
        print(f"  Blue Control: {stats['blue_control']:.2%}")
        print(f"  Red Control: {stats['red_control']:.2%}")


In [ ]:
# データをPandas DataFrameに変換して分析

# プレイヤー位置データのDataFrame作成
player_positions = []
for team in ["blue_team", "red_team"]:
    for player in result[team]:
        player_positions.append({
            "team": "Blue" if team == "blue_team" else "Red",
            "player_id": player["id"],
            "x": player["position"][0],
            "y": player["position"][1],
            "zone": player["zone"]
        })

df_positions = pd.DataFrame(player_positions)
print("\n=== Player Positions DataFrame ===")
print(df_positions)

# ゾーン統計のDataFrame作成
zone_stats = []
for zone, stats in result["zone_stats"].items():
    zone_stats.append({
        "zone": int(zone),
        "blue_players": stats["blue"],
        "red_players": stats["red"],
        "total_players": stats["blue"] + stats["red"]
    })

df_zones = pd.DataFrame(zone_stats)
print("\n=== Zone Statistics DataFrame ===")
print(df_zones)

# 基本的な統計分析
print("\n=== Basic Statistics ===")
print("\nTeam Distribution by Zone:")
print(df_positions.groupby(["team", "zone"]).size().unstack(fill_value=0))

print("\nZone Occupancy Summary:")
print(df_zones.describe())

# 結果をCSVファイルとして保存
df_positions.to_csv("player_positions.csv", index=False)
df_zones.to_csv("zone_statistics.csv", index=False)
print("\nDataFrames saved to CSV files: player_positions.csv and zone_statistics.csv")
